<a href="https://colab.research.google.com/github/AishwaryaGangaVN/codsoft/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
pd.set_option('display.max_columns',None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/creditcard.csv")

In [ ]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [ ]:
df.shape

(11959, 31)

In [ ]:
df.dtypes

Time        int64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class     float64
dtype: object

In [ ]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [ ]:
fraud_count=df['Class'].value_counts()
fraud_rate=100*fraud_count/df.shape[0]
fraud_data=pd.concat([fraud_count,fraud_rate],axis=1).reset_index()
fraud_data.columns=['Class','Count','Percentage']
fraud_data

,Class,Count,Percentage
0,0.0,11906,99.556819
1,1.0,52,0.434819


In [ ]:
df_fraud=df[df['Class']==1]
df_not_fraud=df[df['Class']==0]
df_not_fraud_sampled=df_not_fraud.sample(df_fraud.shape[0],replace=False,random_state=101)
df_balanced=pd.concat([df_not_fraud_sampled,df_fraud],axis=0).sample(frac=1,replace=False,random_state=101).reset_index().drop('index',axis=1)
df_balanced

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8153,0.997434,0.432226,1.536445,2.600987,-0.345564,0.562762,-0.522037,0.170808,0.462303,0.347141,2.701043,-1.063823,2.953685,1.411149,-0.456130,0.859757,-0.057296,0.146797,-1.566954,-0.066061,-0.027377,0.126498,0.083808,0.168859,0.092154,-0.129878,0.014382,0.027168,32.23,0.0
1,532,-0.681072,0.926831,1.871161,0.923397,-0.438215,0.110163,0.918259,-0.222899,-0.040360,0.239503,1.039842,0.386524,-0.983549,-0.061620,-0.602062,-0.580104,-0.077508,0.190753,1.109216,-0.087599,-0.119718,-0.131065,-0.112667,0.518059,-0.330417,-0.581855,-0.363707,-0.111442,75.17,0.0
2,19762,-14.179165,7.421370,-21.405836,11.927512,-7.974281,-2.202710,-15.471612,-0.356595,-6.380125,-13.348278,10.187587,-14.563980,0.593759,-14.491598,-0.281393,-8.264129,-12.439805,-4.837079,0.753150,1.002764,-2.366836,1.130955,0.991153,-1.033132,-0.327179,0.634693,2.171905,-1.395288,1.00,1.0
3,4462,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,-1.525412,2.032912,-6.560124,0.022937,-1.470102,-0.698826,-2.282194,-4.781831,-2.615665,-1.334441,-0.430022,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1.0
4,17230,-0.469327,1.111453,2.041003,1.731595,0.135147,-0.093625,0.266155,0.082988,0.580255,-0.164563,0.500376,-3.021760,0.924213,1.500794,-0.400283,0.706048,0.115114,-0.046796,-2.548522,-0.358709,0.159387,0.592670,-0.053596,0.320748,-0.369121,-0.136605,-0.100845,0.039347,3.93,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,7740,1.023874,2.001485,-4.769752,3.819195,-1.271754,-1.734662,-3.059245,0.889805,0.415382,-3.955812,3.572055,-7.186452,0.147242,-5.249305,1.678334,-2.641473,-1.312059,-0.391716,1.118264,0.204138,0.343283,-0.054196,0.709654,-0.372216,-2.032068,0.366778,0.395171,0.020206,1.00,1.0
100,8528,0.447396,2.481954,-5.660814,4.455923,-2.443780,-2.185040,-4.716143,1.249803,-0.718326,-5.390330,6.454188,-8.485347,0.635281,-7.019902,0.539814,-4.649864,-6.288358,-1.339312,2.262985,0.549613,0.756053,0.140168,0.665411,0.131464,-1.908217,0.334808,0.748534,0.175414,1.00,1.0
101,11635,0.919137,4.199633,-7.535607,7.426940,1.118215,-2.886722,-1.341036,0.363933,-2.203224,-4.137840,4.570113,-7.629170,1.733917,-9.440375,-0.023353,-1.233958,1.632009,1.315735,-0.287189,0.535435,0.316094,0.055179,0.210692,-0.417918,-0.911188,0.466524,0.627393,0.157851,1.00,1.0
102,7140,0.816719,-0.321163,-0.015866,0.799123,-0.168197,-0.708302,0.581961,-0.441462,1.017362,-0.601248,0.547443,-1.701214,3.055363,1.630802,0.133335,0.311525,0.160887,-0.556596,-0.005439,0.469065,-0.517036,-1.826148,-0.024175,-0.170040,0.062929,-0.062135,-0.119186,0.052429,259.90,0.0


In [ ]:
fraud_cont=df_balanced["Class"].value_counts()
fraud_rate= 100*fraud_count/df_balanced.shape[0]
fraud_data=pd.concat([fraud_count,fraud_rate],axis=1).reset_index()
fraud_data.columns=['Class','Count','Percentage']
fraud_data

,Class,Count,Percentage
0,0.0,11906,11448.076923
1,1.0,52,50.000000


In [ ]:
X_train, X_test, y_train, y_test=train_test_split(df_balanced.drop('Class',axis=1),df_balanced['Class'],test_size=0.2,random_state=101)
print(f'''X_train:{X_train.shape}
X_test:{X_test.shape}
y_train:{y_train.shape}
y_test:{y_test.shape}''')

X_train:(83, 30)
X_test:(21, 30)
y_train:(83,)
y_test:(21,)


In [ ]:
randomForestModel= Pipeline([
    ('scaler',StandardScaler()),
    ('Classifier',RandomForestClassifier())
])
randomForestModel.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('Classifier', RandomForestClassifier())])

In [ ]:
y_pred_logis=randomForestModel.predict(X_test)
y_pred_logis

array([0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 0., 1., 0.])

In [ ]:
cr=classification_report(y_test,y_pred_logis)
print(cr)

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96        11
         1.0       1.00      0.90      0.95        10

    accuracy                           0.95        21
   macro avg       0.96      0.95      0.95        21
weighted avg       0.96      0.95      0.95        21

